# Introduction

This notebook was just meant to get used to the environment of PyG.

The models implemented in this notebook aren't ideal and don't perform well.

In the coming tutorials, the implementation of better models can be done on top of this notebook to improve upon its existing performance.

In [1]:
import torch_geometric
from torch_geometric.datasets import Planetoid

# Load the Data

In [2]:
dataset = Planetoid(root='cora-dataset',name='Cora')

Processing...
Done!


#### Dataset Properties

In [3]:
print(dataset)

Cora()


In [4]:
print("Number of graphs:\t\t",len(dataset))
print("Number of classes:\t\t",dataset.num_classes)
print("Number of node features:\t\t",dataset.num_node_features)
print("Number of edge features:\t\t",dataset.num_edge_features)
print(type(dataset))

Number of graphs:		 1
Number of classes:		 7
Number of node features:		 1433
Number of edge features:		 0
<class 'torch_geometric.datasets.planetoid.Planetoid'>


In [5]:
print(dataset.num_features)

1433


##### Dataset Shapes

In [5]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [6]:
print('edge_index:\t\t',dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\n")
print('train_mask:\t\t',dataset.data.train_mask.shape)
print(dataset.data.train_mask)
print("\n")
print('x:\t\t',dataset.data.x.shape)
print(dataset.data.x)
print("\n")
print('y:\t\t',dataset.data.y.shape)
print(dataset.data.y)
print("\n")

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])




In [7]:
import os.path as osp
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import SAGEConv


In [8]:
data = dataset[0]

## Model Definition

Here a basic model was defined with no Linear layers or anything. Just a SAGEConv layer mapping from the number of features to number of classes. 

This will result in barely any improvement in terms of validation and test accuracy

In [9]:
class Net_MAX(nn.Module):
    def __init__(self):
        super(Net_MAX,self).__init__()
        
        self.conv = SAGEConv(dataset.num_features, dataset.num_classes,aggr='max')
        # aggr = max, mean, add ...
    def forward(self):
        x = self.conv(data.x,data.edge_index)
        
        return F.log_softmax(x,dim=1)

In [10]:
class Net_MEAN(nn.Module):
    def __init__(self):
        super(Net_MEAN,self).__init__()
        
        self.conv = SAGEConv(dataset.num_features, dataset.num_classes,aggr='mean')
        # aggr = max, mean, add ...
    def forward(self):
        x = self.conv(data.x,data.edge_index)
        
        return F.log_softmax(x,dim=1)

In [11]:
class Net_ADD(nn.Module):
    def __init__(self):
        super(Net_ADD,self).__init__()
        
        self.conv = SAGEConv(dataset.num_features, dataset.num_classes,aggr='add')
        # aggr = max, mean, add ...
    def forward(self):
        x = self.conv(data.x,data.edge_index)
        
        return F.log_softmax(x,dim=1)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_max, data = Net_MAX().to(device),data.to(device)
model_mean = Net_MEAN().to(device)
model_add = Net_ADD().to(device)

optimizer_max = torch.optim.Adam(model_max.parameters(),lr=0.01,weight_decay=5e-4)
optimizer_mean = torch.optim.Adam(model_mean.parameters(),lr=0.01,weight_decay=5e-4)
optimizer_add = torch.optim.Adam(model_add.parameters(),lr=0.01,weight_decay=5e-4)


##### Definition of a single training / optimizing step

In [13]:
def train(model,optimizer,data):
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask],data.y[data.train_mask]).backward()
    optimizer.step()
    return model
    
def test(model,optimizer,data):
    model.eval()
    logits,accs = model(),[]
    for _,mask in data('train_mask','val_mask','test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

##### Definition of the training loop

In [14]:
def train_loop(model,optimizer,data,epochs):

    best_val_acc = test_acc = 0

    for epoch in range(1,epochs):
        model = train(model,optimizer,data)
        _,val_acc,tmp_test_acc = test(model,optimizer,data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        log = 'Epoch: {}, Val: {:.4f}, Test: {:.4f}'
        
        if epoch%10==0:
            print(log.format(epoch,best_val_acc,test_acc))

#### Network with aggregate function as Max

In [15]:
train_loop(model_max,optimizer_max,data,100)

Epoch: 10, Val: 0.7220, Test: 0.7090
Epoch: 20, Val: 0.7220, Test: 0.7090
Epoch: 30, Val: 0.7220, Test: 0.7090
Epoch: 40, Val: 0.7220, Test: 0.7090
Epoch: 50, Val: 0.7220, Test: 0.7090
Epoch: 60, Val: 0.7220, Test: 0.7090
Epoch: 70, Val: 0.7240, Test: 0.7130
Epoch: 80, Val: 0.7240, Test: 0.7130
Epoch: 90, Val: 0.7240, Test: 0.7130


#### Network with aggregate function as Sum

In [16]:
train_loop(model_add,optimizer_add,data,100)

Epoch: 10, Val: 0.7440, Test: 0.7580
Epoch: 20, Val: 0.7440, Test: 0.7580
Epoch: 30, Val: 0.7440, Test: 0.7580
Epoch: 40, Val: 0.7440, Test: 0.7580
Epoch: 50, Val: 0.7440, Test: 0.7580
Epoch: 60, Val: 0.7440, Test: 0.7580
Epoch: 70, Val: 0.7440, Test: 0.7580
Epoch: 80, Val: 0.7440, Test: 0.7580
Epoch: 90, Val: 0.7440, Test: 0.7580


#### Network with aggregate function as Mean

In [17]:
train_loop(model_mean,optimizer_mean,data,100)

Epoch: 10, Val: 0.7000, Test: 0.7070
Epoch: 20, Val: 0.7020, Test: 0.7070
Epoch: 30, Val: 0.7020, Test: 0.7070
Epoch: 40, Val: 0.7020, Test: 0.7070
Epoch: 50, Val: 0.7020, Test: 0.7070
Epoch: 60, Val: 0.7020, Test: 0.7070
Epoch: 70, Val: 0.7020, Test: 0.7070
Epoch: 80, Val: 0.7020, Test: 0.7070
Epoch: 90, Val: 0.7020, Test: 0.7070
